# Verge and foliot mechanism

References:

- https://github.com/JuliaReach/ReachabilityModels.jl/issues/52

- https://github.com/SciML/DifferentialEquations.jl/issues/703

## Dependencies

In [ ]:
] activate .

In [ ]:
] add LazySets#v1.38.0

In [ ]:
] add ModelingToolkit#v3.20.0

In [ ]:
] add IntervalConstraintProgramming#v0.12.1

In [ ]:
using ReachabilityAnalysis, LinearAlgebra, Plots, IntervalConstraintProgramming, StaticArrays
using BenchmarkTools

const IA = IntervalArithmetic

LazySets.set_ztol(Float64, 1e-12)

gr()

## Model parameters

In [ ]:
const torque = 1.0
const Ic = 10.0
const Iv = 0.15
const rc = 1.0
const rv = 0.3
const αc = 24.0 * pi/180
const er = 0.05
const nteeth = round(Int, 2pi/αc) # n
const Mv = Iv/rv^2
const Mc = Ic/rc^2
const Gc = Mv*(1 + er)/rc/(Mv + Mc)
const Gv = Mc*(1 + er)/rv/(Mv + Mc)
const αv = Gv*rc/(1-er)*αc/2

p = (torque, Ic);

## Guards

In [ ]:
# trigonometric constraints for each m
upper_trig(m) = @constraint $rc*sin(x1 - ($m-1)* ($αc)) - $rv*tan(x2 + $αv/2) == 0
lower_trig(m) = @constraint $rc*sin(($m-1)* ($αc) - x1) - $rv*tan(-x2 + $αv/2) == 0

# domain of x1 for each m
upper_dom_x1(m; p=0) = ((m-1-1/2)*αc .. (m-1+1/2)*αc) + (2p)*pi
lower_dom_x1(m; p=0) = ((m-1-1/2)*αc .. (m-1+1/2)*αc) + (2p-1)*pi

# combined domain of x1 and x2 for each m 
upper_dom(m; p=0) = upper_dom_x1(m, p=p) × (-1 .. 1)
lower_dom(m; p=0) = lower_dom_x1(m, p=p) × (-1 .. 1)

# constraint solver tolerance
ε = 0.05

# upper constraints
Upav = [pave(upper_trig(m), upper_dom(m, p=0), ε).boundary for m in 1:nteeth]
U = [reduce(vcat, vertices_list(B) for B in Ui) |> VPolygon for Ui in Upav]

# lower constraints
Lpav = [pave(lower_trig(m), lower_dom(m, p=0), ε).boundary for m in 1:nteeth];
L = [reduce(vcat, vertices_list(B) for B in Li) |> VPolygon for Li in Lpav];

In [ ]:
fig = plot(xlab="x₁", ylab="x₂", title="Collision conditions, blue: upper, red: lower")

plot!(fig, U, c=:blue)
plot!(fig, L, c=:red)

In [ ]:
G34_upper = HalfSpace(SA[-rc, rv], 0.0)   # rc x3 - rv x4 > 0
G34_lower = HalfSpace(SA[-rc, -rv], 0.0); # rc x3 + rv x4 > 0

## Transition map

In [ ]:
I5 = SA[1 0 0 0 0;
        0 1 0 0 0;
        0 0 1 0 0;
        0 0 0 1 0;
        0 0 0 0 1.]

# σ = 1
Tupper = SA[0 0     0      0   0;
            0 0     0      0   0;
            0 0 -rc*Gc rv*Gc   0;
            0 0 rc*Gv -rv*Gv   0;
            0 0     0      0   0] + I5

# σ = -1
Tlower = SA[0 0     0      0   0;
            0 0     0      0   0;
            0 0 -rc*Gc -rv*Gc  0;
            0 0 -rc*Gv -rv*Gv  0;
            0  0    0      0   0] + I5;

## Continuous post-operator

In [ ]:
# variables x1, x2, x3, x4 and the last variable x5 wraps the constant input term
A = zeros(5, 5)
A[1, 3] = 1.0
A[2, 4] = 1.0
A[3, 5] = 1.0

X0 = Singleton([0, 0, 0, 3.0, p[1]/p[2]])
prob = @ivp(x' = A*x, x(0) ∈ X0);

Timings for different algorithms:

```julia
@btime solve($prob, tspan=(0.0, 0.5), alg=GLGM06(δ=1e-3, static=true, dim=5, ngens=10, max_order=2));
  245.860 μs (229 allocations: 277.58 KiB)
```
    
```julia
@btime solve($prob, tspan=(0.0, 0.5), alg=VREP(δ=1e-3, static=true, dim=5));
  251.422 μs (1940 allocations: 186.80 KiB)
```

```julia
@btime solve($prob, tspan=(0.0, 0.5), alg=VREP(δ=1e-3, static=true, dim=5, backend=CDDLib.Library()));
  116.335 μs (752 allocations: 143.14 KiB)
```

In [ ]:
# system is in dimension 5 and we have to take some concrete set operations in the discretization
using Polyhedra, CDDLib
using LazySets: HalfSpace

In [ ]:
sol = solve(prob, tspan=(0.0, 0.5), alg=VREP(δ=1e-3, static=true, dim=5, backend=CDDLib.Library()));

In [ ]:
plot(sol, vars=(0, 1))

## Discrete post

In [ ]:
F = flowpipe(sol)
typeof(F)

In [ ]:
X = set(F[10])
@which project(X, 1:2)

In [ ]:
X = convert(VPolygon, linear_map(πM12, set(sol[295])))

In [ ]:
L1 = L[1];
L9 = L[9];

In [ ]:
@btime intersection($X, $L1);

In [ ]:
@btime intersection($X, $L9);

In [ ]:
function _check_intersection(X, Li)
    Xbox = box_approximation(X)
    Libox = box_approximation(Li)   # can cache
    return isdisjoint(Xbox, Libox)
end

In [ ]:
@btime _check_intersection($X, $L1)

In [ ]:
@btime _check_intersection($X, $L9)

In [ ]:
[intersection(X, Li) for Li in L]

In [ ]:
SVector{2, Float64}

In [ ]:
const πM12 = SA[1 0 0 0 0;
                0 1 0 0 0.]

# return the indices in the flowpipe fp that intersect any of the guard sets L and U
# in the variables x1 and x2
function _jump_indices(F::Flowpipe{N, <:ReachSet{N, <:VPolytope}}, L::Vector{<:VPolygon}, U::Vector{<:VPolygon}) where {N}
    # return a vector that contains: index of F where intersection happens,
    # index of vector L (or -1 if the intersection is with an element of L),
    # index of vector U (or -1 if the intersection is with an element of U),
    # actualy intersection
    VT = SVector{2, Float64}
    idx = Vector{Tuple{Int, Int, Int, VPolygon{N, VT}}}()

    for (i, R) in enumerate(F)

        # reach-set (polygon) in x1-x2 coordinates 
        X = convert(VPolygon, linear_map(πM12, set(R)))

        for (j, Lj) in enumerate(L)
            Xint_L = intersection(X, Lj)
            if !isempty(Xint_L)
                push!(idx, (i, j, -1, Xint_L))
            end
        end

        # TODO refactor
        for (j, Uj) in enumerate(U)
            Xint_U = intersection(X, Uj)
            if !isempty(Xint_U)
                push!(idx, (i, -1, j, Xint_U))
            end
        end
    end
    return idx
end

In [ ]:
typeof(F)

In [ ]:
@btime _jump_indices_12($F, $L, $U);

In [ ]:
@btime overapproximate($F, $Hyperrectangle);

In [ ]:
# overapproximate(F, Hyperrectangle) # ????

In [ ]:
X = set(F)
@btime box_approximation($X)

In [ ]:
#isdisjoint(overapproximate(F, Hyperrectangle), L[1]) # not implemented

isdisjoint(overapproximate(F, Hyperrectangle), overapproximate(L[1], Hyperrectangle)) # not implemented

In [ ]:
# Idea : speedup with box OA

# return the indices in the flowpipe fp that intersect any of the guard sets L and U
# in the variables x1 and x2
function _jump_indices_3(F::Flowpipe{N, <:ReachSet{N, <:VPolytope}}, L::Vector{<:VPolygon}, U::Vector{<:VPolygon}) where {N}
    # return a vector that contains: index of F where intersection happens,
    # index of vector L (or -1 if the intersection is with an element of L),
    # index of vector U (or -1 if the intersection is with an element of U),
    # actualy intersection
    VT = SVector{2, Float64}
    idx = Vector{Tuple{Int, Int, Int, VPolygon{N, VT}}}()

    # eliminate sets from L and U which do not intersect the flowpipe
    Fbox = overapproximate(set(F), Hyperrectangle) # UnionSetArray(...)

    Lidx = Vector{Int}() 
    for (j, Lj) in enumerate(L)
        Ljbox = overapproximate(Lj, Hyperrectangle)
        println("j = $j")
        if !isdisjoint(Fbox, Ljbox)
            push!(Lidx, j)
        end
    end

    Uidx = Vector{Int}() 
    for (j, Uj) in enumerate(U)
        Ujbox = overapproximate(Uj, Hyperrectangle)
        if !isdisjoint(Fbox, Ujbox)
            push!(Uidx, j)
        end
    end

    for (i, R) in enumerate(F)

        # reach-set (polygon) in x1-x2 coordinates 
        X = convert(VPolygon, linear_map(πM12, set(R)))

        for j in Lidx
            Xint_L = intersection(X, L[j])
            if !isempty(Xint_L)
                push!(idx, (i, j, -1, Xint_L))
            end
        end

        # TODO refactor
        for j in Uidx
            Xint_U = intersection(X, U[j])
            if !isempty(Xint_U)
                push!(idx, (i, -1, j, Xint_U))
            end
        end
    end
    return idx
end

In [ ]:
Fbox = overapproximate(set(F), Hyperrectangle)

Ljbox = overapproximate(L[1], Hyperrectangle)
Ljbox = Hyperrectangle(Vector(Ljbox.center), Vector(Ljbox.radius))

typeof(Fbox)

In [ ]:
typeof(Ljbox)

In [ ]:
Fbox

In [ ]:
Ljbox

In [ ]:
is_intersection_empty(Fbox, Ljbox)

In [ ]:
_jump_indices_3(F, L, U);

In [ ]:
@btime _jump_indices_3($F, $L, $U);

In [ ]:
function cluster_polygon(J::Vector{Tuple{Int, Symbol, VPolygon{N, VT}}}) where {N, VT}
    first_idx = first(J)[1]
    last_idx = last(J)[1]

    vall = Vector{VT}()
    for Ji in J
        for vi in vertices_list(Ji[3])
            push!(vall, vi)
        end
    end
    return VPolygon(vall)
end